In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [3]:
# Path to your labelled dataset
train_dir = r"C:\Users\EMMANUEL AROWESAN\Downloads\archive (1)\CLAHE_images"  # Contains 'benign', 'malignant', 'normal' subfolders

# Path to your unlabelled images folder
unlabelled_dir = r"C:\Users\EMMANUEL AROWESAN\Desktop\imageFolder"  # Replace with your folder name

In [4]:
img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Split into train/validation
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

print(train_generator.class_indices)  # See mapping of labels

Found 21282 images belonging to 3 classes.
Found 5320 images belonging to 3 classes.
{'benign': 0, 'malignant': 1, 'normal': 2}


In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: benign, malignant, normal
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)              

In [6]:
epochs = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

Epoch 1/10


666/666 [==============================] - 1269s 2s/step - loss: 0.5220 - accuracy: 0.7812 - val_loss: 0.6190 - val_accuracy: 0.6998
Epoch 2/10
666/666 [==============================] - 1072s 2s/step - loss: 0.3194 - accuracy: 0.8673 - val_loss: 0.6404 - val_accuracy: 0.6741
Epoch 3/10
666/666 [==============================] - 1140s 2s/step - loss: 0.2464 - accuracy: 0.8965 - val_loss: 0.6916 - val_accuracy: 0.6773
Epoch 4/10
666/666 [==============================] - 1411s 2s/step - loss: 0.1981 - accuracy: 0.9124 - val_loss: 0.7444 - val_accuracy: 0.6782
Epoch 5/10
666/666 [==============================] - 1239s 2s/step - loss: 0.1659 - accuracy: 0.9292 - val_loss: 0.7533 - val_accuracy: 0.6964
Epoch 6/10
666/666 [==============================] - 1312s 2s/step - loss: 0.1347 - accuracy: 0.9448 - val_loss: 0.8224 - val_accuracy: 0.6942
Epoch 7/10
666/666 [==============================] - 991s 1s/step - loss: 0.1113 - accuracy: 0.9563 - val_loss: 0.9801 - val_accuracy

In [7]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    labels = list(train_generator.class_indices.keys())
    return labels[class_index], prediction

# Loop through unlabelled images
for img_file in os.listdir(unlabelled_dir):
    img_path = os.path.join(unlabelled_dir, img_file)
    label, probs = predict_image(img_path)
    print(f"{img_file} → {label} (Confidence: {np.max(probs):.2f})")

1/1 [==============================] - 1s 708ms/step
James Olufunke 51(CC-View).jpg → malignant (Confidence: 1.00)
1/1 [==============================] - 0s 47ms/step
James Olufunke 51(LMO-View).jpg → malignant (Confidence: 0.99)
1/1 [==============================] - 0s 70ms/step
Janehin Abiodun 45 (CC-view).jpg → malignant (Confidence: 1.00)
1/1 [==============================] - 0s 57ms/step
Janehin Abiodun 45 (LMO-view).jpg → benign (Confidence: 0.86)
1/1 [==============================] - 0s 73ms/step
Joe-Cobham Florence 41 P- (CC-view).JPG → malignant (Confidence: 1.00)
1/1 [==============================] - 0s 60ms/step
Joe-Cobham Florence 41 P- (LMO-view).JPG → benign (Confidence: 1.00)
1/1 [==============================] - 0s 84ms/step
Jones Ngozi 41 (CC-view).jpg → malignant (Confidence: 1.00)
1/1 [==============================] - 0s 74ms/step
Jones Ngozi 41 (LMO-view).jpg → malignant (Confidence: 0.92)
1/1 [==============================] - 0s 58ms/step
Kalu Nene 39 P0 (CC

In [8]:
model.save("mammogram_classifier.h5")

C:\Users\EMMANUEL AROWESAN\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
